# InSAR Processing Using ASF HYP3 on-demand products

## Getting Started

The Alaska Satellite Facility (ASF) archives SAR and InSAR data on behalf of NASA. 
By searching on their [Vertex website](https://search.asf.alaska.edu/#/?maxResults=250), users can access and download the original data, or in some cases can run basic processing. 
Vertex also allows users to do basic SBAS processing, which produces interferograms that can be downloaded and processed to time-series. 
This is what we will in this exercise. 


The ASF User guide can be found [here](https://docs.asf.alaska.edu/vertex/manual/)

See the HYP3 SDK [here](https://hyp3-docs.asf.alaska.edu/using/sdk/)

In [1]:
%pip install hyp3_sdk

Note: you may need to restart the kernel to use updated packages.


In [1]:
from pathlib import Path

import numpy as np
import rasterio as rio
import resample_data
from hyp3_sdk import HyP3

In [15]:
# Uses your .netrc credentials for urs.earthdata.nasa.gov.
# See 'InSAR_start.ipynb##Downloading Data'
hyp3 = HyP3()
batch = hyp3.find_jobs()

Once you have complete jobs you can download the products to your machine

In [19]:
print(f'{batch.complete()=}')
print(f'{len(batch)=}')

batch.complete()=True
len(batch)=4


### WARNING!!!
This step will download files from ASF. 
These files vary in size but are roughly ~0.5 GB each, so 200 will be ~100 GB. 
Make sure you have adequate space for downloading before starting! 
The good news is that file downloads don't repeat, so if the code stops for any reason you can restart from where it ended. 

In [2]:
# specify your download folder
dload_path_name = Path('asf_insar_data')

In [20]:
batch.download_files(dload_path_name, create=True)

  0%|          | 0/4 [00:00<?, ?it/s]

S1AA_20250109T231458_20250121T231457_VVP012_INT80_G_ueF_F6AF.zip: 100%|██████████| 203M/203M [00:09<00:00, 22.5MB/s]
S1AA_20250121T231457_20250202T231453_VVP012_INT80_G_ueF_467E.zip: 100%|██████████| 182M/182M [00:09<00:00, 19.2MB/s]
S1AA_20250214T231456_20250226T231456_VVP012_INT80_G_ueF_4AEF.zip: 100%|██████████| 202M/202M [00:09<00:00, 22.3MB/s]
S1AA_20250202T231453_20250214T231456_VVP012_INT80_G_ueF_5EEB.zip: 100%|██████████| 181M/181M [00:09<00:00, 20.8MB/s]
100%|██████████| 4/4 [00:53<00:00, 13.38s/it]


[PosixPath('asf_insar_data/S1AA_20250109T231458_20250121T231457_VVP012_INT80_G_ueF_F6AF.zip'),
 PosixPath('asf_insar_data/S1AA_20250121T231457_20250202T231453_VVP012_INT80_G_ueF_467E.zip'),
 PosixPath('asf_insar_data/S1AA_20250214T231456_20250226T231456_VVP012_INT80_G_ueF_4AEF.zip'),
 PosixPath('asf_insar_data/S1AA_20250202T231453_20250214T231456_VVP012_INT80_G_ueF_5EEB.zip')]

### Steps
The first steps are to download and unzip the products in our working directory. In this case, I have already done this step. 

In [3]:
%ls {dload_path_name}

S1AA_20250109T231458_20250121T231457_VVP012_INT80_G_ueF_F6AF/
S1AA_20250121T231457_20250202T231453_VVP012_INT80_G_ueF_467E/
S1AA_20250202T231453_20250214T231456_VVP012_INT80_G_ueF_5EEB/
S1AA_20250214T231456_20250226T231456_VVP012_INT80_G_ueF_4AEF/
zipped/


In [ ]:
# Unzip your folders
# Depending on your platform this may not work
!unzip -o '{dload_path_name}*.zip'

Archive:  S1AA_20250109T231458_20250121T231457_VVP012_INT80_G_ueF_F6AF.zip
  inflating: S1AA_20250109T231458_20250121T231457_VVP012_INT80_G_ueF_F6AF/S1AA_20250109T231458_20250121T231457_VVP012_INT80_G_ueF_F6AF_lv_theta.tif.xml  
  inflating: S1AA_20250109T231458_20250121T231457_VVP012_INT80_G_ueF_F6AF/S1AA_20250109T231458_20250121T231457_VVP012_INT80_G_ueF_F6AF_amp.tif  
  inflating: S1AA_20250109T231458_20250121T231457_VVP012_INT80_G_ueF_F6AF/S1AA_20250109T231458_20250121T231457_VVP012_INT80_G_ueF_F6AF_lv_phi.tif  
  inflating: S1AA_20250109T231458_20250121T231457_VVP012_INT80_G_ueF_F6AF/S1AA_20250109T231458_20250121T231457_VVP012_INT80_G_ueF_F6AF_color_phase.png.aux.xml  
  inflating: S1AA_20250109T231458_20250121T231457_VVP012_INT80_G_ueF_F6AF/S1AA_20250109T231458_20250121T231457_VVP012_INT80_G_ueF_F6AF.txt  
  inflating: S1AA_20250109T231458_20250121T231457_VVP012_INT80_G_ueF_F6AF/S1AA_20250109T231458_20250121T231457_VVP012_INT80_G_ueF_F6AF_color_phase.png.xml  
  inflating: S1AA_2

In [4]:
%ls {dload_path_name}

S1AA_20250109T231458_20250121T231457_VVP012_INT80_G_ueF_F6AF/
S1AA_20250121T231457_20250202T231453_VVP012_INT80_G_ueF_467E/
S1AA_20250202T231453_20250214T231456_VVP012_INT80_G_ueF_5EEB/
S1AA_20250214T231456_20250226T231456_VVP012_INT80_G_ueF_4AEF/
zipped/


As you can see, we have a number of folders that each contain a set of files, including unwrapped phase, unwrapped displacements, DEM, look vectors, etc. 

Each folder is named by the two dates that were used to generate that interferogram, so for example `S1AA_20220420T235621_20220514T235622_VVP024_INT80_G_ueF_CF44` is an interferogram for dates 4/20/2022 and 5/14/2024. 

In [5]:
# Count the number of interferograms we have downloaded
# If this line does not work on your machine feel free to skip
%ls -ld {dload_path_name}/*/ | wc -l  # this will count folders only

5


In [31]:
%mkdir -p zipped/
%mv *.zip zipped/.
%ls

mv: cannot stat '*.zip': No such file or directory
S1AA_20250109T231458_20250121T231457_VVP012_INT80_G_ueF_F6AF/
S1AA_20250121T231457_20250202T231453_VVP012_INT80_G_ueF_467E/
S1AA_20250202T231453_20250214T231456_VVP012_INT80_G_ueF_5EEB/
S1AA_20250214T231456_20250226T231456_VVP012_INT80_G_ueF_4AEF/
zipped/


In [ ]:
unw_paths = list(dload_path_name.glob('S1*/*_unw_phase.tif'))
corr_paths = list(dload_path_name.glob('S1*/*corr.tif'))

# Some QA/QC:
# - there should be some files in your download dir now.
# - there should be the same number of unw phase files as correlation files.
num_phases = len(unw_paths)
num_cors = len(corr_paths)
print(f'{num_phases=}; {num_cors=}')
assert num_phases > 0
assert num_phases == num_cors

all_widths: list[float] = []
all_heights: list[float] = []
for path in unw_paths:
    with rio.open(path) as f:
        all_widths.append(f.width)
        all_heights.append(f.height)

print(f'Average width: {np.mean(all_widths)}')
print(f'Average height: {np.mean(all_heights)}')

num_phases=4; num_cors=4
Average width: 3632.5
Average height: 2787.0


The next step is to make sure that all the interferograms have the same shape and geographic extent. 
We will ensure this with a program that uses rasterio to re-project all the interferograms to match a reference. 
For this code, you will need `rasterio` and `numpy` installed. 

In [23]:
# for k, f in enumerate(files):
#     with rio.open(f) as F:
#         plt.plot([k,k], F.

In [ ]:
# The function to resize the images is called run_resampling
shapefile_path = Path('~/CEF/Projects/SLU/t92f120/asf_insar_data/stl_polygon/stl_polygon.shp')
assert shapefile_path.exists()
resample_data.main(shapefile_path=shapefile_path)  # runs on the current directory by default

AssertionError: 

Once the resampling has finished, we can now proceed with time-series analysis as normal!
You can refer to the [InSAR lab](https://github.com/jlmaurer/GE6146/blob/master/notebooks/Lab4_InSAR.ipynb) for more details and plotting examples. 

In [ ]:
# We'll use the same functions as for the lab
import make_ts

In [ ]:
dir = Path.cwd()
ifg_paths = list(dir.glob('**/*_unw_phase.tif'))
coh_paths = list(dir.glob('**/*_corr.tif'))

In [ ]:
print(f'I found {len(ifg_paths)} interferograms')

I found 771 interferograms


In [ ]:
# Pull all the data
date_pairs, dates = make_ts.get_dates(ifg_paths)
year_fracs = np.array([make_ts.get_year_frac(d) for d in dates])
g_matrix = make_ts.make_g_matrix(dates, date_pairs)
x_size, y_size, data_type, geo_proj, transform, nodata_val, num_bands = make_ts.get_raster_metadata(ifg_paths[0])
data = make_ts.get_data(ifg_paths, 1)

In [ ]:
print(f'Interferograms are {y_size}x{x_size} pixels in size')

Interferograms are 2841x3557 pixels in size


In [ ]:
# You'll need to make a plot of one of the dates or the average coherence so you can pick a reference point

In [ ]:
# De-referencing
data = make_ts.dereference(data, ref_center=(2500, 2000), ref_size=10)

In [ ]:
# Create the time-series data cube
ts_array = make_ts.make_ts(g_matrix, data)

In [ ]:
# Convert to displacement in meters
ts_array = make_ts.radians_to_meters(ts_array, wavelength_m=0.056)  # wavelength is 5.6 cm for Sentinel-1

In [ ]:
# Compute the mean velocity by fitting a line
vel = make_ts.find_mean_vel(ts_array, year_fracs)

In [ ]:
vel.shape

(2841, 3557)

### Saving Data
Want to change this to use NETCDF and xarray, include time-series and coherence time-series in the files, maybe IFGs and coherence ifgs as well. 

In [ ]:
import h5py


with h5py.File('velocity.h5') as path:
    for key in path.keys():
        print(key)

intercept
interceptStd
residue
velocity
velocityStd


## Tropospheric correction using RAiDER

TBD